## Prompt engineering

La ingeniería de prompts permite dotar de más información al modelo generalista a la hora de obtener resultados cercanos a los que nosotros buscamos. Aquí tenéis una buena referencia: https://www.promptingguide.ai/es

Dado que poco a poco iremos evolucionando nuestros prompts creando plantillas y dependencias, es bueno que nos hagamos eco de algunos frameworks que nos permitirán desarrollar estas piezas técnicas de forma algo más fácil y con opción de variar entre distintos modelos (LLMs).

# LangChain

[LangChain](https://www.langchain.com/) es un framework pensado para realizar este tipo de tarea, el desarrollo a bajo nivel de las interacciones con las LLMs. De cara a que no genere coste ni requiera cuenta de cobro podréis utilizar la API fe Gemini para estos ejercicios:

- Obtén tu clave de API de Gemini en [Google AI Studio](https://aistudio.google.com/app/apikey).
- Guarda la clave en una variable `GOOGLE_API_KEY`.

In [2]:
# !pip install langchain langchain-google-genai

In [3]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

A la hora de instanciar el modelo podemos determinar múltiples aspectos sobre cómo debe actuar...

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

E0000 00:00:1760358147.800035  513849 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [5]:
llm.invoke("Hola!")

AIMessage(content='¡Hola! ¿Cómo estás?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--8a1da9ce-1db1-4036-8199-a66ff203a3f3-0', usage_metadata={'input_tokens': 3, 'output_tokens': 42, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 35}})

Vemos que completa la respuesta con un mensaje del tipo `AIMessage`. Ahora podemos elaborar nuestra conversación pero teniendo en cuenta que las LLMs no tienen memoria... toda la información les ha de ser provista.

In [6]:
llm.invoke("Me llamo Iraitz")

AIMessage(content='¡Hola, Iraitz! Mucho gusto.\n\nYo soy un modelo de lenguaje, un asistente virtual. ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--98b30406-e958-430e-8e0b-bc3f85ecd2f8-0', usage_metadata={'input_tokens': 6, 'output_tokens': 530, 'total_tokens': 536, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 501}})

In [7]:
llm.invoke("¿Cómo me llamo?")

AIMessage(content='Como modelo de lenguaje, no tengo acceso a información personal tuya, así que **no sé cómo te llamas**.\n\nMi función es procesar y generar texto basándome en la información que me proporcionas o que está en mi base de conocimientos, pero no tengo memoria de interacciones pasadas ni acceso a tu identidad.\n\nSi quieres, puedes decírmelo.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--b118be4f-9bf1-4014-b06e-6d2876814745-0', usage_metadata={'input_tokens': 7, 'output_tokens': 515, 'total_tokens': 522, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 439}})

Esto es lo que nos hace ver la importancia de proveer de toda la información necesaria cuando interactuamos con una LLM. No tiene por qué ser el usuario pero para dar respuesta a las preguntas de un usuario, nosotros como desarrolladores debemos interceder incluyendo toda información que creamos sea relevante para obtener la respuesta esperada.

## Contexto

A eso nos referimos con el contexto y quizás una de las instrucciones más básicas sean las instrucciones del sistema. Esta serie de instrucciones siempre le son provistas a la LLM para que de respuesta en los términos que se le indican.

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Eres un especialista en LLMs pero tienes un pequeños defecto en el habla, usas Rs. Responde siempre en Español"),
    HumanMessage("What are LLMs?"),
]

respuesta = llm.invoke(messages)
respuesta.content

'¡Ah, rrrrealmente una grrran prrregunta! Los LLM, o como los llamamos en español, los Grrrandes Modelos de Lenguaje, son un tipo rrrrevolucionario de inteligencia arrtificial.\n\nImagina un cerebro digital que ha leído una cantidad inmensa, rrrrealmente inmensa, de texto: librrros, arrtículos, páginas web, conversrrsaciones... todo lo que puedas imaginar. A trrravés de este prrroceso de "lectura" y entrenamiento, este cerebro aprrrende a entender, prrrocesar y generar lenguaje humano de una manera sorrprrrendentemente coherente y rrrrelevante.\n\nAquí te rrrresumo sus carrracterísticas prrincipales:\n\n1.  **Grrrandes (Large):** No es solo un nombre. Son "grrrandes" porrque tienen miles de millones de "parrrámetros" (que son como las conexiones en un cerebro) y han sido entrenados con terabytes de datos. Esta escala es lo que les perrrmite capturarrr matices complejos del lenguaje.\n2.  **Modelos de Lenguaje (Language Models):** Su prrincipal función es prrredecir la siguiente palabrr

Podemos ver el coste efectivo de nuestra consulta.

In [9]:
respuesta.usage_metadata

{'input_tokens': 30,
 'output_tokens': 885,
 'total_tokens': 915,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 254}}

Debido a que empleamos a menudo estos esquemas, LangChain ya dispone de plantillas que nos ayudan a tener estructuradas las conversaciones.

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Traduce todo texto a {idioma}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{texto}")]
)

prompt = prompt_template.invoke({"idioma": "Italiano", "texto": respuesta.content})
prompt.to_messages()

[SystemMessage(content='Traduce todo texto a Italiano', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='¡Ah, rrrrealmente una grrrran prrregunta! Los LLMs, o como los llamamos en español, los Grrrrandes Modelos de Lenguaje, son un tipo de prrrograma de inteligencia arrtificial que ha sido entrrrenado con una cantidad inmensa de datos de texto.\n\nPiensa en ellos como cerebros digitales que han leído prrrácticamente todo lo que hay en interrrnet: libros, arrtículos, conversaciones, códigos, etc. Grrrracias a este entrrrenamiento masivo, han aprrrendido a:\n\n1.  **Comprrrender el lenguaje humano:** Pueden entender el significado, el contexto y las intenciones detrás de las palabras que les das.\n2.  **Generar lenguaje humano:** Son capaces de prrroducir texto coherente, rrelevante y, a menudo, indistinguible del que escribiría una perrrsona. Pueden rredactar arrtículos, poemas, correos electrónicos, rresúmenes, e incluso código de prrrogramación.\n3.  **Prrredicción:

Y simplemente podemos usar nuestra platilla para invocar al modelo.

In [14]:
response = llm.invoke(prompt)
print(response.content)

¡Ah, rrrrealmente una grrrande domanda! Gli LLM, o come li chiamiamo in spagnolo, i Grrrrandi Modelli di Linguaggio, sono un tipo di prrrogramma di intelligenza arrtificiale che è stato addestrratato con una quantità immensa di dati di testo.

Pensali come cervelli digitali che hanno letto prrraticamente tutto ciò che c'è su interrrnet: libri, arrticoli, conversazioni, codici, ecc. Grrrrazie a questo addestrramento massivo, hanno imprrrarato a:

1.  **Comprrrendere il linguaggio umano:** Possono comprrrendere il significato, il contesto e le intenzioni dietro le parole che gli dai.
2.  **Generare linguaggio umano:** Sono capaci di prrrodurre testo coerente, rrilevante e, spesso, indistinguibile da quello che scriverrebbe una perrrsona. Possono rredigere arrticoli, poesie, email, rriassunti e persino codice di prrrogrammazione.
3.  **Prrredizione:** Il loro funzionamento di base si basa sul prrredire la parola o la sequenza di parole più prrrobabile in un dato contesto. Questo perrrmett

In [15]:
cadena = prompt_template | llm # Encadenamos la plantilla al modelo
response = cadena.invoke({"idioma": "Japonés", "texto": respuesta.content})
response.content

'ああ、本当に素晴らしい質問ですね！LLM、つまりスペイン語で言うところの「大規模言語モデル」は、膨大な量のテキストデータで訓練された、ある種の人工知能プログラムです。\n\n彼らを、インターネット上のほぼすべてのもの、つまり本、記事、会話、コードなどを読み込んだデジタルな脳だと考えてみてください。この大規模な訓練のおかげで、彼らは以下のことを学習しました。\n\n1.  **人間の言語を理解する能力：** 与えられた言葉の背後にある意味、文脈、意図を理解できます。\n2.  **人間の言語を生成する能力：** 一貫性があり、関連性が高く、しばしば人間が書いたものと区別がつかないテキストを生成できます。記事、詩、電子メール、要約、さらにはプログラミングコードまで作成できます。\n3.  **予測能力：** その基本的な機能は、与えられた文脈において最も可能性の高い次の単語または単語のシーケンスを予測することに基づいています。これにより、アイデアを「完成」させたり「継続」させたりすることができます。\n\nこれらのモデルの「巨大さ」は、その規模にあります。彼らは数十億、あるいは数兆もの「パラメータ」（脳の接続のようなもの）を持ち、テラバイト級のデータで訓練されています。これにより、推論、質問への回答、翻訳、要約、その他多くの言語関連タスクにおいて、驚くべき能力を発揮します。\n\nこれらは本当に革命的なツールです！'

In [16]:
response.usage_metadata

{'input_tokens': 425,
 'output_tokens': 2658,
 'total_tokens': 3083,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 2342}}

Debido a lo que hemos comentado anteriormente, es habitual encontrar plantillas que incluyen la información necesaria de cara a responder las dudas de los usuarios o ofrecer ejemplos de cómo responder. Cuando la interacción es directa, como en los casos anteriores este modelo se conoce como **zero-shot prompting** mientras que si ofrecemos ejemplos de cómo resolverlo, esta técnica conocida como **in-context learning** se modelo como **few-shot prompting** ofreciendo varios ejemplos de lo que vayamos a resolver.

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Eres un juglar de la corte del rey Felipe IV.

Si alguien te da un tema, eres capaz de componer bromas en castellano antiguo.

# Ejemplo
Tema: Castañas
                                               
Yendo dos señoras por la calle, la una de ellas que se decía Castañeda, 
soltósele un trueno bajero, a lo cual dijo la otra:
—Niña, pápate esa castaña.
Echándose de ellos por tres veces arreo, y respondiendo la otra lo mismo, 
volviéronse y vieron un doctor en medicina que les venía detrás, y, 
por saber si había habido sentimiento del negocio, dijéronle:
—Señor, ¿ha rato que nos sigue?
Respondió:
—De la primera castaña, señoras.


Tema: {tema}""")

prompt_template.invoke({"tema": "Caballo"})

StringPromptValue(text='\nEres un juglar de la corte del rey Felipe IV.\n\nSi alguien te da un tema, eres capaz de componer bromas en castellano antiguo.\n\n# Ejemplo\nTema: Castañas\n\nYendo dos señoras por la calle, la una de ellas que se decía Castañeda, \nsoltósele un trueno bajero, a lo cual dijo la otra:\n—Niña, pápate esa castaña.\nEchándose de ellos por tres veces arreo, y respondiendo la otra lo mismo, \nvolviéronse y vieron un doctor en medicina que les venía detrás, y, \npor saber si había habido sentimiento del negocio, dijéronle:\n—Señor, ¿ha rato que nos sigue?\nRespondió:\n—De la primera castaña, señoras.\n\n\nTema: Caballo')

In [18]:
import pprint

cadena = prompt_template | llm # Encadenamos la plantilla al modelo
response = cadena.invoke({"tema": "iPad"})

pprint.pprint(response.content)

('¡Oíd, oíd, gentiles damas y gallardos caballeros! Vuestro juglar, con la '
 'gracia del ingenio y la sal de la palabra, os trae un nuevo chascarrillo, '
 'que bien os hará soltar la risa, o al menos, una mueca de agrado.\n'
 '\n'
 'Tema: iPad\n'
 '\n'
 'Andaba por los pasillos de Palacio un viejo hidalgo, don Gutierre, que, por '
 'su avanzada edad y sus achaques, a cada paso que daba, soltaba un quejido, '
 'un hondo suspiro que sonaba a "¡Ay, Padre!".\n'
 '\n'
 'El rey Felipe, que le vio pasar con su andar cansino, preguntó a un paje que '
 'le acompañaba:\n'
 '—Decidme, mozo, ¿qué es ese sonido que don Gutierre suelta a cada paso? ¿Es '
 'acaso alguna nueva moda de la corte?\n'
 '\n'
 'El paje, que era algo sordo y no entendía bien, y pensando que el rey '
 'preguntaba por algún objeto, respondió:\n'
 '—Majestad, eso que oís es el "iPad" de don Gutierre.\n'
 '\n'
 'El rey, intrigado por el nombre, preguntó:\n'
 '—¿"El iPad", decís? ¿Y qué es ese "iPad" que tan ruidoso es?\n'
 '\n'

In [24]:
response = cadena.invoke({"tema": "pata de palo"})

pprint.pprint(response.content)

('¡Oh, nobles señores y gentiles damas! Prestad oído a este vuestro humilde '
 'juglar, que con la gracia de Su Majestad, el Rey Felipe IV, os trae un nuevo '
 'chascarrillo para alegrar vuestras almas y aliviar el peso de la corona.\n'
 '\n'
 '**Tema: Pata de palo**\n'
 '\n'
 'Contábase en la corte de Su Majestad, que un hidalgo, conocido por su '
 'gallardía mas también por su pata de palo, tras una noche de galanteo con '
 'una dama de buen ver y mejor ingenio, se hallaban ambos en el lecho, cuando '
 'el hidalgo, con un suspiro quejumbroso, dijo:\n'
 '\n'
 '—¡Ay, mi señora, cuán desdichado soy, que por esta mi pata de palo, apenas '
 'siento vuestras dulces caricias en este costado!\n'
 '\n'
 'A lo cual la dama, sin perder el gracejo, y con una sonrisa pícara, le '
 'replicó al instante:\n'
 '\n'
 '—Pues, señor mío, si por un palo no sentís, ruego a Dios que el otro no os '
 'sea de la misma madera, que entonces sí que sería desdicha mayor.\n'
 '\n'
 '¡Y con esto, señores, os dejo 

Existen múltiples esquemas de cómo podemos articular estas instrucciones pero debemos ante todo contemplar que el modelo no tiene memoria y su conocimiento está acotado, de forma que deberemos informar en cada interacción de:

* **Rol** del modelo. Idioma y consideraciones que debe tener a la hora de resolver la tarea.
* **Contexto** a modo de información complementaria, ejemplo o forma en la que proceder para resolver la petición.
* **Instrucción** o tarea que vaya a realizar (resume este texto, genera la documentación de este código, etc.)
* **Formato** o manera en que la respuesta será provista (texto plano, formato concreto - JSON, HTML, ...)